In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import seaborn as sns

# Total population, N.
N = 126e2
# Initial number of infected and recovered individuals, I0 and R0.
I0, Q0, R0 = 117328, 84419, 0
# Everyone else, S0, is susceptible to infection initially.
S0 = N - I0 - R0 - Q0
# Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
beta, gamma = 1./6, 1./15 
# A grid of time points (in days)
t = np.linspace(0, 365, 365)
# The SIR model differential equations.
Ntest = 1e6
f = 30.0
posrate = 0.8
hosprate = 0.20

def deriv(y, t, N, beta, gamma):
    S, I, Q, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - Ntest* f*posrate* I/N - gamma * I
    dQdt = Ntest* f*posrate* I/N - gamma * Q
    dRdt = gamma * (I+Q)
    return dSdt, dIdt, dQdt, dRdt

# Initial conditions vector
y0 = S0, I0, Q0, R0

# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, Q, R = ret.T
print('max hosp {}'.format(np.max(hosprate*(I+Q)/N)))

# Plot the data on three separate curves for S(t), I(t) and R(t)
fig, axarr = plt.subplots(3, 1, sharex=True, figsize=(9,7)) #facecolor='w')
fs = 12

# plot total cases
axarr[0].plot(t, R/N*100, 'g', alpha=0.5, lw=2, label='Cumulated Cases')
axarr[1].set_ylabel('% of Population', fontsize=fs)
axarr[0].set_ylim(0,33)
legend = axarr[0].legend(loc=4, fontsize=fs)
# plot infected
axarr[1].plot(t, I/N*100, 'r', alpha=0.5, lw=2, label='Infected and active')
#axarr[1].plot(t, Q/N*100, 'm', alpha=0.5, lw=2, label='Infected and quarantined')
axarr[0].set_ylabel('% of Population', fontsize=fs)
axarr[1].set_ylim(0,1.6)
axarr[1].legend(loc=1, fontsize=fs)
# hospitalized
axarr[2].plot(t, hosprate*(I+Q)/N*100, 'b', linestyle='-', alpha=0.5, lw=2, label='Hospitalized')
axarr[2].set_xlabel('Time (days)', fontsize=fs)
axarr[2].set_ylabel('% of Population', fontsize=fs)
axarr[2].legend(loc=1, fontsize=fs)
axarr[2].set_ylim(0,.7)
axarr[2].axvline(np.argmax(hosprate*(I+Q)/N*100), linestyle='--', color='k')
axarr[2].axhline(np.max(hosprate*(I+Q)/N*100), linestyle='--', color='k', label = 'max load')
axarr[2].legend(loc='center right', fontsize=fs)
fig.suptitle('Results with 30m naive tests or 1m targeted tests (f=30)', fontsize=18)
fig.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.show()

def maxhosp(Ntest, f=1.0):
    # Total population, N.
    N = 126e2
    # Initial number of infected and recovered individuals, I0 and R0.
    I0, Q0, R0 = 117328, 84419, 0
    # Everyone else, S0, is susceptible to infection initially.
    S0 = N - I0 - R0 - Q0
    # Contact rate, beta, and mean recovery rate, gamma, (in 1/days).
    beta, gamma = 1./6.0, 1.0/15.0 
    # A grid of time points (in days)
    t = np.linspace(0, 365, 365)
    # The SIR model differential equations.
    posrate = 0.6
    hosprate = 0.20
    def deriv(y, t, N, beta, gamma):
        S, I, Q, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - Ntest* f*posrate* I/N - gamma * I
        dQdt = Ntest* f*posrate* I/N - gamma * Q
        dRdt = gamma * (I+Q)
        return dSdt, dIdt, dQdt, dRdt
    
    # Initial conditions vector
    y0 = S0, I0, Q0, R0
    
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    S, I, Q, R = ret.T
    return np.max(hosprate*(I+Q)/N)

ntests = np.logspace(3, np.log10(20e6), 300.0)
sims1 = [maxhosp(n, f=1.0) for n in ntests]
sims3 = [maxhosp(n, f=3.0) for n in ntests]
sims10 = [maxhosp(n, f=10.0) for n in ntests]
sims30 = [maxhosp(n, f=30.0) for n in ntests]

# plot in frac of population
fig, ax = plt.subplots(figsize=(8,6))
ax.axhline(1, color='r', lw=4, linestyle='-', label='#beds in Mexico')
ax.plot(ntests/1e6, np.array(sims1)*N/1e6, lw=3, label='$f=1.0$')
#ax.plot(ntests/1e6, np.array(sims3)*N/1e6, lw=3, label='$f=3.0$')
ax.plot(ntests/1e6, np.array(sims10)*N/1e6, lw=3, label='$f=10.0$')
ax.plot(ntests/1e6, np.array(sims30)*N/1e6, lw=3, label='$f=30.0$', color='m')
ax.set_ylabel('Peak Hospitalized in Millions', fontsize=fs)
ax.set_xlabel('Tests in Millions per Day', fontsize=fs)
ax.set_title('Peak hospitalization depends on\nnumber of tests and how targeted they are', fontsize=18)
ax.legend(fontsize=fs, loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlim((0,10))
ax.set_ylim((0,16))
plt.show()

max hosp 3.2023333333333333


<Figure size 900x700 with 3 Axes>

<Figure size 800x600 with 1 Axes>